傍示サンプルをデフォルトにして、足していく

hist, new, total, new/hist で特徴量作成

https://www.kaggle.com/code/raddar/target-true-meaning-revealed


amountを調整

np.round(df['purchase_amount'] / 0.00150265118 + 497.06)

In [1]:
import warnings
import time
import sys
import datetime
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

%matplotlib inline

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def preprocessing(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    for col in ['category_3']:
        df[col] = df[col].map({'A':1, 'B':2,'C':3})
    return df

def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
    return df

In [10]:
# データの読み込み
path_new = '../../../../data/raw/new_merchant_transactions.csv'
path_hist = '../../../../data/raw/historical_transactions.csv'
path_train = '../../../../data/raw/train.csv'
path_test = '../../../../data/raw/test.csv'

new = pd.read_csv(path_new, parse_dates=['purchase_date'])
hist = pd.read_csv(path_hist ,parse_dates=['purchase_date'])

new= preprocessing(new)
hist = preprocessing(hist)

#new = reduce_mem_usage(new)
#hist = reduce_mem_usage(hist)

train = pd.read_csv(path_train, parse_dates=['first_active_month'])
test = pd.read_csv(path_test, parse_dates=['first_active_month'])

target = train['target']
#del train['target']

In [37]:
new_0 = pd.read_csv(path_new, parse_dates=['purchase_date'])
new_0 = preprocessing(new_0)

In [21]:
train = pd.read_csv(path_train, parse_dates=['first_active_month'])
test = pd.read_csv(path_test, parse_dates=['first_active_month'])

## train, test 処理

In [22]:
#train['target'] = 2**train['target']

In [15]:
test_null_id = test[test.first_active_month.isna()].card_id.iloc[-1]
null_month = hist[hist.card_id == test_null_id].purchase_date.dt.strftime("%Y-%m-01").min()
test.fillna({'first_active_month':null_month},inplace=True)

In [16]:
for df in [test, train]:
    # https://www.kaggle.com/code/mfjwr1/simple-lightgbm-without-blending
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
    df['quarter'] = df['first_active_month'].dt.quarter
    df['days_feature1'] = df['elapsed_time'] * df['feature_1']
    df['days_feature2'] = df['elapsed_time'] * df['feature_2']
    df['days_feature3'] = df['elapsed_time'] * df['feature_3']
    df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
    df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
    df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']
    df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
    df['feature_mean'] = df['feature_sum']/3
    df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
    df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
    df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

In [17]:
for df in [test,train]:
    df['first_active_month']  = (df['first_active_month'] - df['first_active_month'].min()).dt.days

## 特徴量作成

In [18]:
for df in [hist, new]:
    df['purchase_amount'] = np.round(df['purchase_amount'] / 0.00150265118 + 497.06)

In [19]:
hist['month_diff'] = ((datetime.datetime.today() - hist['purchase_date']).dt.days)//30
hist['month_diff'] += hist['month_lag']

new['month_diff'] = ((datetime.datetime.today() - new['purchase_date']).dt.days)//30
new['month_diff'] += new['month_lag']


historical_transactions = pd.get_dummies(hist, columns=['category_2', 'category_3'])
new_transactions = pd.get_dummies(new, columns=['category_2', 'category_3'])

historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)

agg_fun = {'authorized_flag': ['mean']}
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]

historical_transactions['purchase_month'] = historical_transactions['purchase_date'].dt.month
authorized_transactions['purchase_month'] = authorized_transactions['purchase_date'].dt.month
new_transactions['purchase_month'] = new_transactions['purchase_date'].dt.month

Mem. usage decreased to 1332.66 Mb (57.1% reduction)
Mem. usage decreased to 89.86 Mb (57.1% reduction)


In [24]:
def aggregate_transactions(history):
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).astype(np.int64) * 1e-9
    
    agg_func = {
    'category_1': ['sum', 'mean'],
    'category_2_1.0': ['mean'],
    'category_2_2.0': ['mean'],
    'category_2_3.0': ['mean'],
    'category_2_4.0': ['mean'],
    'category_2_5.0': ['mean'],
    'category_3_1.0': ['mean'],
    'category_3_2.0': ['mean'],
    'category_3_3.0': ['mean'],
    'merchant_id': ['nunique'],
    'merchant_category_id': ['nunique'],
    'state_id': ['nunique'],
    'city_id': ['nunique'],
    'subsector_id': ['nunique'],
    'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
    'installments': ['sum', 'mean', 'max', 'min', 'std'],
    'purchase_month': ['mean', 'max', 'min', 'std'],
    'purchase_date': [np.ptp, 'min', 'max'],
    'month_lag': ['mean', 'max', 'min', 'std'],
    'month_diff': ['mean']
    }
    
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

In [28]:
history = aggregate_transactions(historical_transactions)
history.columns = ['history_' + c if c != 'card_id' else c for c in history.columns]
history[:5]

,card_id,history_transactions_count,history_category_1_sum,history_category_1_mean,history_category_2_1.0_mean,history_category_2_2.0_mean,history_category_2_3.0_mean,history_category_2_4.0_mean,history_category_2_5.0_mean,history_category_3_1.0_mean,history_category_3_2.0_mean,history_category_3_3.0_mean,history_merchant_id_nunique,history_merchant_category_id_nunique,history_state_id_nunique,history_city_id_nunique,history_subsector_id_nunique,history_purchase_amount_sum,history_purchase_amount_mean,history_purchase_amount_max,history_purchase_amount_min,history_purchase_amount_std,history_installments_sum,history_installments_mean,history_installments_max,history_installments_min,history_installments_std,history_purchase_month_mean,history_purchase_month_max,history_purchase_month_min,history_purchase_month_std,history_purchase_date_ptp,history_purchase_date_min,history_purchase_date_max,history_month_lag_mean,history_month_lag_max,history_month_lag_min,history_month_lag_std,history_month_diff_mean
0,C_ID_00007093c1,35,4,0.114286,0.000000,0.000000,0.885714,0.0,0.000000,0.000000,0.828571,0.171429,11,8,2,3,7,7812.0,223.199997,1500.0,12.0,286.205788,45,1.285714,3,1,0.667367,5.914286,12,1,3.071419,0.0,1.000000e-09,1.000000e-09,-6.028571,0,-11,3.535415,76.971429
1,C_ID_0001238066,3,0,0.000000,0.333333,0.000000,0.000000,0.0,0.666667,0.000000,1.000000,0.000000,2,1,2,2,1,300.0,100.000000,100.0,100.0,0.000000,3,1.000000,1,1,0.000000,8.666667,12,2,5.773503,0.0,1.000000e-09,1.000000e-09,-1.333333,0,-2,1.154701,76.333333
2,C_ID_0001506ef0,4,0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.750000,0.250000,0.000000,4,4,1,1,4,55.0,13.750000,26.0,4.0,10.965856,1,0.250000,1,0,0.500000,5.500000,12,2,4.725816,0.0,1.000000e-09,1.000000e-09,-2.500000,0,-8,3.785939,77.000000
3,C_ID_0001793786,27,2,0.074074,0.111111,0.296296,0.111111,0.0,0.000000,0.814815,0.185185,0.000000,17,14,4,8,11,5553.0,205.666672,1320.0,1.0,331.490225,5,0.185185,1,0,0.395847,6.481481,9,3,2.375684,0.0,1.000000e-09,1.000000e-09,-3.518519,-1,-7,2.375684,80.925926
4,C_ID_000183fdda,7,0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.428571,0.571429,4,4,1,1,4,2852.0,407.428558,719.0,22.0,301.004904,19,2.714286,6,1,2.288689,8.285714,10,8,0.755929,0.0,1.000000e-09,1.000000e-09,-5.714286,-4,-6,0.755929,76.571429


In [26]:
authorized = aggregate_transactions(authorized_transactions)
authorized.columns = ['auth_' + c if c != 'card_id' else c for c in authorized.columns]
authorized[:5]

,card_id,auth_transactions_count,auth_category_1_sum,auth_category_1_mean,auth_category_2_1.0_mean,auth_category_2_2.0_mean,auth_category_2_3.0_mean,auth_category_2_4.0_mean,auth_category_2_5.0_mean,auth_category_3_1.0_mean,auth_category_3_2.0_mean,auth_category_3_3.0_mean,auth_merchant_id_nunique,auth_merchant_category_id_nunique,auth_state_id_nunique,auth_city_id_nunique,auth_subsector_id_nunique,auth_purchase_amount_sum,auth_purchase_amount_mean,auth_purchase_amount_max,auth_purchase_amount_min,auth_purchase_amount_std,auth_installments_sum,auth_installments_mean,auth_installments_max,auth_installments_min,auth_installments_std,auth_purchase_month_mean,auth_purchase_month_max,auth_purchase_month_min,auth_purchase_month_std,auth_purchase_date_ptp,auth_purchase_date_min,auth_purchase_date_max,auth_month_lag_mean,auth_month_lag_max,auth_month_lag_min,auth_month_lag_std,auth_month_diff_mean
0,C_ID_00007093c1,114,24,0.210526,0.000000,0.000000,0.780702,0.0,0.008772,0.0,0.842105,0.157895,28,18,3,4,13,15108.0,132.526321,1300.0,12.0,158.005445,147,1.289474,6,1,0.795159,6.517544,12,1,3.371490,32627654.0,1.487081e+09,1.519708e+09,-5.798246,0,-12,3.441495,76.938596
1,C_ID_0001238066,120,2,0.016667,0.783333,0.000000,0.000000,0.0,0.150000,0.0,0.708333,0.266667,65,29,6,18,17,12625.0,105.208336,1008.0,8.0,128.184668,195,1.625000,10,-1,1.501050,7.275000,12,1,4.895483,13110825.0,1.506638e+09,1.519748e+09,-1.825000,0,-5,1.294218,76.958333
2,C_ID_0001506ef0,62,0,0.000000,0.032258,0.000000,0.967742,0.0,0.000000,1.0,0.000000,0.000000,28,19,2,3,12,9724.0,156.838715,1491.0,4.0,322.480560,0,0.000000,0,0,0.000000,6.887097,12,1,4.538017,34460275.0,1.484411e+09,1.518871e+09,-4.983871,0,-13,4.248402,77.000000
3,C_ID_0001793786,189,0,0.000000,0.042328,0.359788,0.063492,0.0,0.000000,1.0,0.000000,0.000000,114,45,4,9,22,77338.0,409.195770,3528.0,6.0,577.597812,0,0.000000,0,0,0.000000,6.698413,10,1,2.301491,24487497.0,1.484994e+09,1.509481e+09,-3.301587,0,-9,2.301491,80.968254
4,C_ID_000183fdda,137,4,0.029197,0.051095,0.007299,0.905109,0.0,0.007299,0.0,0.729927,0.240876,71,34,7,9,20,22915.0,167.262772,2337.0,6.0,348.570448,245,1.788321,10,-1,2.108912,6.810219,12,1,4.538289,15148616.0,1.504444e+09,1.519592e+09,-2.284672,0,-5,1.782055,76.897810


In [27]:
new_agg = aggregate_transactions(new_transactions)
new_agg.columns = ['new_agg_' + c if c != 'card_id' else c for c in new_agg.columns]
new_agg[:5]

,card_id,new_transactions_count,new_category_1_sum,new_category_1_mean,new_category_2_1.0_mean,new_category_2_2.0_mean,new_category_2_3.0_mean,new_category_2_4.0_mean,new_category_2_5.0_mean,new_category_3_1.0_mean,new_category_3_2.0_mean,new_category_3_3.0_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_mean,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_month_diff_mean
0,C_ID_00007093c1,2,0,0.000000,0.500000,0.000000,0.50000,0.0,0.000000,0.0,1.000000,0.000000,2,2,2,2,2,110.0,55.000000,60.0,50.0,7.071068,2,1.000000,1,1,0.000000,4.000000,4,4,0.000000,537024.0,1.522754e+09,1.523291e+09,2.000000,2,2,0.000000,77.000000
1,C_ID_0001238066,26,2,0.076923,0.769231,0.000000,0.00000,0.0,0.115385,0.0,0.807692,0.153846,25,15,4,8,9,3042.0,117.000000,445.0,4.0,115.406066,42,1.615385,10,-1,2.079941,3.346154,4,3,0.485165,5195343.0,1.519923e+09,1.525118e+09,1.346154,2,1,0.485165,76.807692
2,C_ID_0001506ef0,2,0,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,1.0,0.000000,0.000000,1,2,1,1,2,31.0,15.500000,21.0,10.0,7.778175,0,0.000000,0,0,0.000000,3.000000,3,3,0.000000,471152.0,1.521239e+09,1.521710e+09,1.000000,1,1,0.000000,77.000000
3,C_ID_0001793786,31,0,0.000000,0.483871,0.258065,0.16129,0.0,0.032258,1.0,0.000000,0.000000,31,21,5,7,14,15255.0,492.096771,2580.0,6.0,630.386620,0,0.000000,0,0,0.000000,11.322581,12,11,0.475191,3981096.0,1.510761e+09,1.514742e+09,1.322581,2,1,0.475191,80.806452
4,C_ID_000183fdda,11,0,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,0.0,0.545455,0.363636,11,9,2,2,6,1081.0,98.272728,425.0,10.0,121.540192,16,1.454545,4,-1,1.293340,3.272727,4,3,0.467099,5106807.0,1.519994e+09,1.525100e+09,1.272727,2,1,0.467099,76.909091


In [29]:
def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_lag'])

    agg_func = {
            'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group
#___________________________________________________________
final_group =  aggregate_per_month(authorized_transactions) 
final_group[:10]

,card_id,month_lag_mean,month_lag_std,purchase_amount_count_mean,purchase_amount_count_std,purchase_amount_sum_mean,purchase_amount_sum_std,purchase_amount_mean_mean,purchase_amount_mean_std,purchase_amount_min_mean,purchase_amount_min_std,purchase_amount_max_mean,purchase_amount_max_std,purchase_amount_std_mean,purchase_amount_std_std,installments_count_mean,installments_count_std,installments_sum_mean,installments_sum_std,installments_mean_mean,installments_mean_std,installments_min_mean,installments_min_std,installments_max_mean,installments_max_std,installments_std_mean,installments_std_std
0,C_ID_00007093c1,-6.000000,3.894440,8.769231,3.539158,1162.153809,517.478155,140.256042,50.161773,29.461538,27.485194,412.769226,296.926241,131.344521,79.991500,8.769231,3.539158,11.307692,4.210792,1.369646,0.423190,1.000000,0.000000,2.538462,1.613246,0.625428,0.675607
1,C_ID_0001238066,-2.500000,1.870829,20.000000,11.696153,2104.166748,1539.143322,95.602760,30.665598,11.000000,1.788854,389.833344,323.282797,100.745856,62.444976,20.000000,11.696153,32.500000,24.501020,1.652873,0.410539,0.666667,0.816497,4.666667,2.875181,1.256525,0.521292
2,C_ID_0001506ef0,-6.230769,4.225988,4.769231,4.342692,748.000000,580.154003,199.788681,180.489261,14.076923,7.376331,604.384644,498.303880,330.282160,276.229106,4.769231,4.342692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,C_ID_0001793786,-4.500000,3.027650,18.900000,12.160501,7733.799805,4417.661543,493.327637,180.550613,66.900002,110.670130,1955.199951,859.034704,585.830191,172.420772,18.900000,12.160501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,C_ID_000183fdda,-2.500000,1.870829,22.833333,7.167054,3819.166748,2656.495542,171.489304,93.845609,12.333333,6.592926,1263.500000,707.395717,314.208741,209.940306,22.833333,7.167054,40.833333,14.048725,1.834943,0.362557,0.333333,1.032796,9.166667,2.041241,2.144389,0.665529
5,C_ID_00024e244b,-6.384615,4.330867,4.076923,4.290717,324.538452,847.342081,41.067421,47.853034,11.923077,10.602370,214.000000,541.702871,72.217330,138.903571,4.076923,4.290717,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,C_ID_0002709b5a,-5.000000,3.316625,6.090909,3.505839,369.181824,168.165881,72.037231,40.495178,31.090910,20.393404,133.636368,61.610507,50.826986,34.407375,6.090909,3.505839,10.727273,8.427229,1.965729,1.287108,0.818182,0.603023,3.818182,3.341203,1.600058,1.734256
7,C_ID_00027503e2,-4.000000,2.738613,2.666667,1.224745,11.333333,4.949747,4.601852,1.600661,3.222222,1.986063,6.111111,1.900292,2.053910,0.821941,2.666667,1.224745,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,C_ID_000298032a,-5.250000,4.334249,3.500000,1.772811,512.375000,865.924764,104.349701,111.811875,40.250000,13.625082,357.250000,750.559363,142.403628,294.599710,3.500000,1.772811,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,C_ID_0002ba3c2e,-4.666667,3.141125,9.166667,4.665476,668.333313,461.892484,72.476845,29.974284,17.000000,17.481419,239.500000,148.096928,71.481375,40.481275,9.166667,4.665476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [30]:
def successive_aggregates(df, field1, field2):
    t = df.groupby(['card_id', field1])[field2].mean()
    u = pd.DataFrame(t).reset_index().groupby('card_id')[field2].agg(['mean', 'min', 'max', 'std'])
    u.columns = [field1 + '_' + field2 + '_' + col for col in u.columns.values]
    u.reset_index(inplace=True)
    return u

In [31]:
additional_fields = successive_aggregates(new_transactions, 'category_1', 'purchase_amount')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'installments', 'purchase_amount'), on = 'card_id', how='left')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'city_id', 'purchase_amount'), on = 'card_id', how='left')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'category_1', 'installments'), on = 'card_id', how='left')

In [32]:
additional_fields

,card_id,category_1_purchase_amount_mean,category_1_purchase_amount_min,category_1_purchase_amount_max,category_1_purchase_amount_std,installments_purchase_amount_mean,installments_purchase_amount_min,installments_purchase_amount_max,installments_purchase_amount_std,city_id_purchase_amount_mean,city_id_purchase_amount_min,city_id_purchase_amount_max,city_id_purchase_amount_std,category_1_installments_mean,category_1_installments_min,category_1_installments_max,category_1_installments_std
0,C_ID_00007093c1,55.000000,55.000000,55.000000,NaN,55.000000,55.000000,55.000000,NaN,55.000000,50.000000,60.000000,7.071068,1.000000,1.000000,1.000000,NaN
1,C_ID_0001238066,92.937500,64.500000,121.375000,40.216698,215.123810,80.619049,396.000000,147.703388,125.622917,53.333332,183.000000,48.373737,2.479167,1.458333,3.500000,1.443676
2,C_ID_0001506ef0,15.500000,15.500000,15.500000,NaN,15.500000,15.500000,15.500000,NaN,15.500000,15.500000,15.500000,NaN,0.000000,0.000000,0.000000,NaN
3,C_ID_0001793786,492.096771,492.096771,492.096771,NaN,492.096771,492.096771,492.096771,NaN,332.552399,116.000000,663.000000,230.835773,0.000000,0.000000,0.000000,NaN
4,C_ID_000183fdda,98.272728,98.272728,98.272728,NaN,169.966660,33.833332,425.000000,155.479384,116.150002,94.300003,138.000000,30.900564,1.454545,1.454545,1.454545,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289996,C_ID_ffff1d9928,62.500000,12.000000,113.000000,71.417785,160.833328,14.666667,307.000000,206.710882,62.500000,12.000000,113.000000,71.417785,2.500000,1.000000,4.000000,2.121320
289997,C_ID_ffff579d3a,55.000000,55.000000,55.000000,NaN,55.000000,55.000000,55.000000,NaN,55.000000,55.000000,55.000000,NaN,0.000000,0.000000,0.000000,NaN
289998,C_ID_ffff756266,749.000000,749.000000,749.000000,NaN,749.000000,749.000000,749.000000,NaN,749.000000,749.000000,749.000000,NaN,-1.000000,-1.000000,-1.000000,NaN
289999,C_ID_ffff828181,84.125000,84.125000,84.125000,NaN,117.666664,64.000000,186.000000,62.308373,97.277779,44.666668,186.000000,49.647386,1.000000,1.000000,1.000000,NaN


In [38]:
for df in [hist, new]:
    df['category_2'].fillna(1,inplace=True) # 1が最頻値
    df['category_3'].fillna(1,inplace=True) # 1が最頻値
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True) # 最頻値

    # https://www.kaggle.com/code/mfjwr1/simple-lightgbm-without-blending
    df['installments'].replace(-1, np.nan,inplace=True)
    df['installments'].replace(999, np.nan,inplace=True)
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    df['hour'] = df['purchase_date'].dt.hour
    #df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['weekday'] = df['purchase_date'].dt.weekday
    df['weekend'] = (df['purchase_date'].dt.weekday >=5).astype(int)
    # additional features
    df['price'] = df['purchase_amount'] / df['installments']
    # additional features
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['duration'] = df['purchase_amount']*df['month_diff']
    df['amount_month_ratio'] = df['purchase_amount']/df['month_diff']

In [39]:
# 承認フラグ　Yes,Noの数と比
# histのみ（newはYesのみのため）
authorized_flag = hist[['card_id','authorized_flag']].groupby(['card_id','authorized_flag']).agg(count=('card_id','count')).reset_index()

# カードIDごとの合計を計算
total_counts = authorized_flag.groupby('card_id')['count'].sum().reset_index()
total_counts = total_counts.rename(columns={'count': 'total_count'})

# 元のデータと合計を結合
authorized_flag = authorized_flag.merge(total_counts, on='card_id')

# 割合を計算
authorized_flag['ratio'] = authorized_flag['count'] / authorized_flag['total_count']
#hist[['card_id','authorized_flag']].groupby(['card_id']).agg(count=('authorized_flag','mean')).reset_index()

# 結果を整形
authorized_ratio = authorized_flag.pivot(index='card_id', columns='authorized_flag', values=['ratio','count']).reset_index().rename(columns={0: 'No', 1: 'Yes'})
authorized_ratio.columns = ['_'.join(col).strip() for col in authorized_ratio.columns.values]
authorized_ratio.rename(columns={'card_id_': 'card_id'},inplace=True)

In [40]:
total = pd.concat([hist,new],axis=0).reset_index(drop=True)

In [41]:
hist_id_u = hist.groupby('card_id',as_index=False)\
    .agg(
        frequency = ('card_id','count'),
        amount_total = ('purchase_amount','sum'),
        amount_mean = ('purchase_amount','mean'),
        amount_max = ('purchase_amount','max'),
        amount_min = ('purchase_amount','min'),
        amount_median = ('purchase_amount','median'),
        merchant_category_nu = ('merchant_category_id','nunique'),
        merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
        city_nu = ('city_id','nunique'),
        city_mode = ('city_id',lambda x: x.mode()[0]),
        state_nu = ('state_id','nunique'),
        state_mode = ('state_id',lambda x: x.mode()[0]),
        date_min  = ('purchase_date','min'),
        date_max  = ('purchase_date','max'),
        installments_mean = ('installments','mean'),
        installments_max = ('installments','max'),
        installments_min = ('installments','min'),
        installments_median = ('installments','median'),
        installments_sum = ('installments','sum'),
        month_lag_mean = ('month_lag','mean'),
        month_lag_max = ('month_lag','max'),
        month_lag_min = ('month_lag','min'),
        month_lag_median = ('month_lag','median'),
        category_1_mod = ('category_1',lambda x: x.mode()[0]),
        category_2_mod = ('category_2',lambda x: x.mode()[0]),
        category_3_mod = ('category_3',lambda x: x.mode()[0]),
        # additional
        weekend_mean = ('weekend', 'mean'),
        weekday_mean = ('weekday', 'mean'),
        day_nu = ('day', 'nunique'),
        day_min = ('day', 'min'),
        day_max = ('day', 'max'),
        price_sum = ('price', 'sum'),
        price_min = ('price', 'min'),
        price_max = ('price', 'max'),
        price_mean = ('price', 'mean'),
        duration_min = ('duration', 'min'),
        duration_max = ('duration', 'max'),
        duration_mean = ('duration', 'mean'),
        amount_month_ratio_min = ('amount_month_ratio', 'min'),
        amount_month_ratio_max = ('amount_month_ratio', 'max'),
        amount_month_ratio_mean = ('amount_month_ratio', 'mean')
    )

In [42]:
hist_id_u = hist_id_u.merge(authorized_ratio,on='card_id')

In [43]:
hist_id_u['duration'] = (hist_id_u['date_max'] - hist_id_u['date_min']).dt.days
hist_id_u.date_max = (hist_id_u.date_max - hist_id_u.date_min.min()).dt.days
hist_id_u.date_min = (hist_id_u.date_min - hist_id_u.date_min.min()).dt.days

hist_id_u.merchant_category_mode = pd.to_numeric(hist_id_u.merchant_category_mode, errors='coerce')

hist_id_u.columns = ['hist_' + col if col != 'card_id' else col for col in hist_id_u.columns]

In [44]:
new_id_u = new.groupby('card_id',as_index=False)\
    .agg(
        frequency = ('card_id','count'),
        amount_total = ('purchase_amount','sum'),
        amount_mean = ('purchase_amount','mean'),
        amount_max = ('purchase_amount','max'),
        amount_min = ('purchase_amount','min'),
        amount_median = ('purchase_amount','median'),
        merchant_category_nu = ('merchant_category_id','nunique'),
        merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
        city_nu = ('city_id','nunique'),
        city_mode = ('city_id',lambda x: x.mode()[0]),
        state_nu = ('state_id','nunique'),
        state_mode = ('state_id',lambda x: x.mode()[0]),
        date_min  = ('purchase_date','min'),
        date_max  = ('purchase_date','max'),
        installments_mean = ('installments','mean'),
        installments_max = ('installments','max'),
        installments_min = ('installments','min'),
        installments_median = ('installments','median'),
        installments_sum = ('installments','sum'),
        month_lag_mean = ('month_lag','mean'),
        month_lag_max = ('month_lag','max'),
        month_lag_min = ('month_lag','min'),
        month_lag_median = ('month_lag','median'),
        category_1_mod = ('category_1',lambda x: x.mode()[0]),
        category_2_mod = ('category_2',lambda x: x.mode()[0]),
        category_3_mod = ('category_3',lambda x: x.mode()[0]),
        # additional
        weekend_mean = ('weekend', 'mean'),
        weekday_mean = ('weekday', 'mean'),
        day_nu = ('day', 'nunique'),
        day_min = ('day', 'min'),
        day_max = ('day', 'max'),
        price_sum = ('price', 'sum'),
        price_min = ('price', 'min'),
        price_max = ('price', 'max'),
        price_mean = ('price', 'mean'),
        duration_min = ('duration', 'min'),
        duration_max = ('duration', 'max'),
        duration_mean = ('duration', 'mean'),
        amount_month_ratio_min = ('amount_month_ratio', 'min'),
        amount_month_ratio_max = ('amount_month_ratio', 'max'),
        amount_month_ratio_mean = ('amount_month_ratio', 'mean')
    )

In [45]:
new_id_u['duration'] = (new_id_u['date_max'] - new_id_u['date_min']).dt.days
new_id_u.date_max = (new_id_u.date_max - new_id_u.date_min.min()).dt.days
new_id_u.date_min = (new_id_u.date_min - new_id_u.date_min.min()).dt.days

new_id_u.merchant_category_mode = pd.to_numeric(new_id_u.merchant_category_mode, errors='coerce')

new_id_u.columns = ['new_' + col if col != 'card_id' else col for col in new_id_u.columns]

In [46]:
total_id_u = total.groupby('card_id',as_index=False)\
    .agg(
        frequency = ('card_id','count'),
        amount_total = ('purchase_amount','sum'),
        amount_mean = ('purchase_amount','mean'),
        amount_max = ('purchase_amount','max'),
        amount_min = ('purchase_amount','min'),
        amount_median = ('purchase_amount','median'),
        merchant_category_nu = ('merchant_category_id','nunique'),
        merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
        city_nu = ('city_id','nunique'),
        city_mode = ('city_id',lambda x: x.mode()[0]),
        state_nu = ('state_id','nunique'),
        state_mode = ('state_id',lambda x: x.mode()[0]),
        date_min  = ('purchase_date','min'),
        date_max  = ('purchase_date','max'),
        installments_mean = ('installments','mean'),
        installments_max = ('installments','max'),
        installments_min = ('installments','min'),
        installments_median = ('installments','median'),
        installments_sum = ('installments','sum'),
        month_lag_mean = ('month_lag','mean'),
        month_lag_max = ('month_lag','max'),
        month_lag_min = ('month_lag','min'),
        month_lag_median = ('month_lag','median'),
        category_1_mod = ('category_1',lambda x: x.mode()[0]),
        category_2_mod = ('category_2',lambda x: x.mode()[0]),
        category_3_mod = ('category_3',lambda x: x.mode()[0]),
        # additional
        weekend_mean = ('weekend', 'mean'),
        weekday_mean = ('weekday', 'mean'),
        day_nu = ('day', 'nunique'),
        day_min = ('day', 'min'),
        day_max = ('day', 'max'),
        price_sum = ('price', 'sum'),
        price_min = ('price', 'min'),
        price_max = ('price', 'max'),
        price_mean = ('price', 'mean'),
        duration_min = ('duration', 'min'),
        duration_max = ('duration', 'max'),
        duration_mean = ('duration', 'mean'),
        amount_month_ratio_min = ('amount_month_ratio', 'min'),
        amount_month_ratio_max = ('amount_month_ratio', 'max'),
        amount_month_ratio_mean = ('amount_month_ratio', 'mean')
    )

In [47]:
total_id_u['duration'] = (total_id_u['date_max'] - total_id_u['date_min']).dt.days
total_id_u.date_max = (total_id_u.date_max - total_id_u.date_min.min()).dt.days
total_id_u.date_min = (total_id_u.date_min - total_id_u.date_min.min()).dt.days

total_id_u.merchant_category_mode = pd.to_numeric(total_id_u.merchant_category_mode, errors='coerce')

total_id_u.columns = ['total_' + col if col != 'card_id' else col for col in total_id_u.columns]

In [48]:
# データの結合
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')

train = pd.merge(train, authorized, on='card_id', how='left')
test = pd.merge(test, authorized, on='card_id', how='left')

train = pd.merge(train, new_agg, on='card_id', how='left')
test = pd.merge(test, new_agg, on='card_id', how='left')

train = pd.merge(train, final_group, on='card_id', how='left')
test = pd.merge(test, final_group, on='card_id', how='left')

train = pd.merge(train, auth_mean, on='card_id', how='left')
test = pd.merge(test, auth_mean, on='card_id', how='left')

train = pd.merge(train, additional_fields, on='card_id', how='left')
test = pd.merge(test, additional_fields, on='card_id', how='left')



In [49]:
train = train.merge(hist_id_u, on='card_id',how='left')
test = test.merge(hist_id_u, on='card_id',how='left')

train = train.merge(new_id_u, on='card_id',how='left')
test = test.merge(new_id_u, on='card_id',how='left')

train = train.merge(total_id_u, on='card_id',how='left')
test = test.merge(total_id_u, on='card_id',how='left')

In [36]:
# for df in [train, test]:
#     df['hist_count_No'].fillna(0,inplace=True)
#     df['hist_ratio_No'].fillna(0,inplace=True)
#     df['new_amount_total'].fillna(0,inplace=True)
#     df['new_amount_min'].fillna(0,inplace=True)
#     df['new_amount_max'].fillna(0,inplace=True)
#     df['new_amount_mean'].fillna(0,inplace=True)
#     df['new_amount_median'].fillna(0,inplace=True)
#     df['new_merchant_category_nu'].fillna(0,inplace=True)
#     df['new_city_nu'].fillna(0,inplace=True)
#     df['new_state_nu'].fillna(0,inplace=True)
#     df['new_installments_sum'].fillna(0,inplace=True)
#     df['new_installments_min'].fillna(0,inplace=True)
#     df['new_installments_max'].fillna(0,inplace=True)
#     df['new_installments_mean'].fillna(0,inplace=True)
#     df['new_installments_median'].fillna(0,inplace=True)
#     df['new_price_sum'].fillna(0,inplace=True)
#     df['new_price_min'].fillna(0,inplace=True)
#     df['new_price_max'].fillna(0,inplace=True)
#     df['new_price_mean'].fillna(0,inplace=True)
#     #df.fillna(0, inplace=True)

In [51]:
for df in [train, test]:
    df['ratio_amount_total'] = df['new_amount_total'] / df['hist_amount_total']
    df['ratio_amount_mean'] = df['new_amount_mean'] / df['hist_amount_mean']
    df['ratio_merchant_category_nu'] = df['new_merchant_category_nu'] / df['hist_merchant_category_nu']
    #df['ratio_installments_mean'] = df['new_installments_mean'] / df['hist_installments_mean']
    #df['ratio_installments_sum'] = df['new_installments_sum'] / df['hist_installments_sum']

In [52]:
path_train_prepro = '../../../../data/processed/train_processed_40.csv'
path_test_preprp = '../../../../data/processed/test_processed_40.csv'

train.to_csv(path_train_prepro,index=False)
test.to_csv(path_test_preprp,index=False)